In [63]:
import cmipld, glob, json, os, re
from rdflib import Graph as RGraph
from rdflib import Namespace, URIRef
prefix = cmipld.prefix()

prefix

'emd'

In [ ]:
CONTEXTS = glob.glob('*/_context')


In [68]:
for cloc in CONTEXTS:
    folder = cloc.replace('_context','').rstrip('/')
    ctx = json.load(open(cloc))
    vocab = ctx['@context']["@vocab"]
    base = ctx['@context']["@base"]
    
    ids = sorted(os.popen(f'jq -r \'.["@id"]\' {folder}/*.json | grep -v null').read().splitlines())
    print(ids)
    
    ## JSONLD
    graph = {**ctx, "@type": ["Collection"],"graph": [{"@id":i} for i in ids]}
    
    with open(f'{folder}/_graph.jsonld','w') as f:
        f.write(re.sub(r'\{\s+"@id":\s+"([^"]+)"\s+\}', r'{"@id": "\1"}',json.dumps(graph,indent=2)))
      
    ## RDF
    data = cmipld.expand(graph,depth=2)
    rdfG = RGraph()


    # Bind prefixes to the graph
    rdfG.bind("_", Namespace(vocab))
    rdfG.bind(f"{prefix}:{folder}", Namespace(base))
    # rdfG.bind("emd", emd_vocab)

    rdfG.parse(data=json.dumps(data[vocab+'graph']),
                base=vocab,
                format="json-ld")

    # Save to a local file (Turtle format is common)
    rdfG.serialize(destination=f"{folder}/_graph.ttl", format="turtle")
    # g2.parse("graph.ttl", format="turtle")     
    # pause    # 
    break
    

['access', 'arpege-climat', 'bcc-csm', 'bisicles', 'cam', 'canesm', 'cesm', 'clm', 'cnrm-cm', 'ec-earth', 'fgoals', 'gelato', 'geos', 'gfdl-cm4', 'giss-e2', 'hadam', 'hadcm2', 'hadgem3', 'icon', 'ifs', 'ipsl-cm', 'miroc', 'mpi-esm', 'nemo', 'nicam', 'pisces', 'reprobus', 'surfex', 'tactic', 'ukesm1']
[Cache HIT] {'@context': {'@alt_base': 'https://wcrp-cmip.github.io/Essential-Model-Documentation/model_family/', '@base': 'https://emd.mipcvs.dev/model_family/', '@vocab': 'https://emd.mipcvs.dev/docs/contents/ModelFamily/', 'collaborative_institutions': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'primary_institution': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'references': {'@context': 'https://emd.mipcvs.dev/reference/_context', '@type': '@id'}, 'scientific_domains': {'@context': 'https://universal.mipcvs.dev/scientific_domain/_context', '@type': '@id'}}, '@type': ['Collection'], 'graph': [{'@id': 'access

In [72]:
# Function to shorten URIs
def shorten(uri):
    if isinstance(uri, URIRef):
        try:
            return rdfG.namespace_manager.qname(uri)
        except:
            return str(uri)
    return str(uri)

# Now print with shortened names
for s, p, o in rdfG:
    print(f"{shorten(s)} -- {shorten(p)} --> {shorten(o)}")

emd:model_family:gfdl-cm4 -- _:scientific_domains --> ns1:ocean
emd:model_family:tactic -- _:scientific_domains --> ns1:aerosol
emd:model_family:ipsl-cm -- rdf:type --> ns2:model_family
emd:model_family:hadcm2 -- _:description --> HadCM2 is an early family of Met Office Hadley Centre coupled climate models that established foundational approaches for UK climate modeling.
emd:model_family:hadgem3 -- _:scientific_domains --> ns1:land_surface
emd:model_family:clm -- _:documentation --> https://escomp.github.io/ctsm-docs/
emd:model_family:cam -- _:description --> Community Atmosphere Model - the atmospheric component of CESM, also used standalone for atmospheric research.
emd:model_family:mpi-esm -- _:primary_institution --> ns3:mpi-m
emd:model_family:arpege-climat -- rdf:type --> _:emd
emd:model_family:miroc -- _:shared_code_base --> none
emd:model_family:hadgem3 -- _:primary_institution --> ns3:mohc
emd:model_family:hadgem3 -- _:validation_key --> HadGEM3
emd:model_family:surfex -- _:val

In [ ]:
# Print all namespaces in the graph
for prefix, namespace in rdfG.namespaces():
    print(f"{prefix}: {namespace}")

[Cache HIT] {'@context': {'@alt_base': 'https://wcrp-cmip.github.io/Essential-Model-Documentation/model_family/', '@base': 'https://emd.mipcvs.dev/model_family/', '@vocab': 'https://emd.mipcvs.dev/docs/contents/ModelFamily/', 'collaborative_institutions': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'primary_institution': {'@context': 'https://universal.mipcvs.dev/organisation/_context', '@type': '@id'}, 'references': {'@context': 'https://emd.mipcvs.dev/reference/_context', '@type': '@id'}, 'scientific_domains': {'@context': 'https://universal.mipcvs.dev/scientific_domain/_context', '@type': '@id'}}, '@type': ['Collection'], 'graph': [{'@id': 'access'}, {'@id': 'arpege-climat'}, {'@id': 'bcc-csm'}, {'@id': 'bisicles'}, {'@id': 'cam'}, {'@id': 'canesm'}, {'@id': 'cesm'}, {'@id': 'clm'}, {'@id': 'cnrm-cm'}, {'@id': 'ec-earth'}, {'@id': 'fgoals'}, {'@id': 'gelato'}, {'@id': 'geos'}, {'@id': 'gfdl-cm4'}, {'@id': 'giss-e2'}, {'@id': 'hadam'}, {'@id': 

dict_keys(['@type', 'https://emd.mipcvs.dev/docs/contents/ModelFamily/graph'])